In [1]:
%load_ext autoreload
%autoreload 2

import logging
from pathlib import Path
from warnings import simplefilter

import pandas as pd
from dotenv import dotenv_values

from generator_drainage_units import run_generator_order_levels

simplefilter(action="ignore", category=pd.errors.PerformanceWarning)
logging.basicConfig(level=logging.DEBUG)

In [2]:
# Define case and base_dir

config = dotenv_values("..\\.env")
base_dir = config["BASE_DIR"]
waterschap = "Waterschap Vallei & Veluwe"
# case_name = "vallei_en_veluwe"
# case_name = "geerestein"
# case_name = "hattemerbroek"
# case_name = "pangelerbeek"
case_name = "Leuvenumse_beek"

# Define waterboard specific variables
range_order_code_min = 721
range_order_code_max = 760

In [ ]:
%%time
case_path = Path(base_dir, case_name)
order = run_generator_order_levels(
    path=case_path,
    waterschap=waterschap,
    range_order_code_min=range_order_code_min,
    range_order_code_max=range_order_code_max,
    water_lines=["hydroobjecten"],
    read_results=True,
    write_results=True,
    create_html_map=False,
    open_html=False
)

In [ ]:
order_labels = False if order.name == "vallei_en_veluwe" else True
order.generate_folium_map(base_map="Light Mode", order_labels=order_labels);
# order.folium_map

In [5]:
nodes = order.nodes.copy()
edges = order.edges.copy()
outflow_nodes = order.outflow_nodes.copy()

nodes = nodes.merge(outflow_nodes[["nodeID", "order_no"]].rename(columns={"order_no": "outflow_node_order_no"}), how="left", on="nodeID")
nodes["outflow_node_order_no"] = nodes["outflow_node_order_no"].fillna(-999).astype(int)

edges = edges.merge(outflow_nodes[["nodeID", "order_no"]].rename(columns={"order_no": "outflow_node_order_no", "nodeID": "node_end"}), how="left", on="node_end")
edges["outflow_node_order_no"] = edges["outflow_node_order_no"].fillna(-999).astype(int)
edges = edges.sort_values(["rws_code", "rws_code_no", "order_no", "outflow_node", "order_edge_no"])

edges_orders = None
for order_no in edges.order_no.unique():
    edges_order = edges[edges.order_no==order_no].copy()
    edges_next_order = edges[edges.order_no==order_no+1].copy()
    edges_order = edges_order.merge(
        edges_next_order[["code", "node_end", "order_no"]].rename(columns={
            "code": "edge_code", "order_no": "outflow_node_order_no"
        }), 
        how='left', 
        on=["node_end", "outflow_node_order_no"]
    )
    # display(edges_order)
    if edges_orders is None:
        edges_orders = edges_order.copy()
    else:
        edges_orders = pd.concat([edges_orders, edges_order]).copy()
# display(edges_orders)

edges_orders = edges_orders.sort_values(
    ["node_end", "outflow_node_order_no"]
).groupby("node_end").agg({
    k: 'count' if k == 'edge_code' else "first" for k in edges_orders.columns
}).sort_values(["rws_code", "rws_code_no", "order_no", "outflow_node", "order_edge_no"])#.reset_index()

In [ ]:
import geopandas as gpd

edges_outflow_nodes_all = None

for order_no in range(2, 10):
    print(order_no)
    order_outflow_nodes = order.outflow_nodes[
        (order.outflow_nodes.rws_code=="VE") &
        (order.outflow_nodes.rws_code_no==728) &
        (order.outflow_nodes.order_no==order_no)
    ]
    display(order_outflow_nodes)

    edges_outflow_nodes = None
    for i_outflow_node, node in order_outflow_nodes.iterrows():
        print(node.order_code)
        edges_outflow_node = edges_orders[
            (edges_orders.outflow_node==node.nodeID)
        ].copy()
        if edges_outflow_node.empty:
            continue
        edges_outflow_node["order_code_no"] = edges_outflow_node["edge_code"]
        edges_outflow_node.loc[edges_outflow_node.order_code_no>0, "order_code_no"] = edges_outflow_node.loc[edges_outflow_node.order_code_no>0, "order_code_no"] + 1
        edges_outflow_node.loc[edges_outflow_node.index[0], "order_code_no"] = 1
        edges_outflow_node["order_code_no"] = edges_outflow_node["order_code_no"].cumsum()
        
        def generate_order_code(x):
            return f"{x["rws_order_code"]}.{str(x["order_code_no"]).zfill(3)}"
        
        edges_outflow_node["order_code"] = edges_outflow_node.apply(lambda x: generate_order_code(x), axis=1)
        display(edges_outflow_node)
        if edges_outflow_nodes is None:
            edges_outflow_nodes = edges_outflow_node.copy()
        else:
            edges_outflow_nodes = pd.concat([edges_outflow_nodes, edges_outflow_node]).copy()
            
    if edges_outflow_nodes_all is None:
        edges_outflow_nodes_all = edges_outflow_nodes.copy()
    else:
        edges_outflow_nodes_all = pd.concat([edges_outflow_nodes_all, edges_outflow_nodes]).copy()

edges_outflow_nodes_all = gpd.GeoDataFrame(
    edges_outflow_nodes_all,
    geometry="geometry",
    crs=28992
).reset_index(drop=True)

edges_outflow_nodes_all.to_file(Path(order.dir_inter_results, "edges.gpkg"), layer="edges")